# Exercice semaine 3

Perform the following tasks using the dataframe (containing all protected areas) that you created last week with the Vahatra data: 

1. Using statsmodels, fit an OLS model with the deforestation rate over the period 2000 - 2023 as your response variable and the year of creation of the protected area as your predictor variable. Design your matrix of predictors "manually" using the DataFrame method of pandas. Print a summary of the results and discuss the sign, magnitude and statistical significance of the coefficient on deforest_2000_2023. 
2. Train a similar OLS model using the ModelSpec method of the ISLP package to design your matrix of predictors.
3. Generate predictions for Corridor Forestier Ambositra-Vondrozo, Mantadia, Menabe Antimena, and Montagne d'Ambre/Forêt d'Ambre, using the OLS model that you just fitted. Create an array or a dataframe to generate a table which compares the predicted values and the true values of the deforestation rate for these four protected areas.
4. Compute confidence intervals and prediction intervals for these four areas and print them.